### Pytorch introduction TLDR

A very simple pytorch CNN classifier.

**Author:** `Anguelos nicolaou anguelos.nikolaou@fau.de`

#### 3 Principal componentnts of a Pytorch train program:

* Define Model class (or use a predefined one)
* Define Dataset class (or use a predefined one)
* Define Train loop:
 * **Define Loss**
 * **Initialise Optimiser**
 * For epoch in training:
  * For batch in epoch:
   * **optimizer.zero_grad()**: Empty gradient buffers
   * **compute loss**: Compute forward pass
   * **loss.backward()**: Compute backward pass
   * **optimizer.step()**: Apply gradient buffers on model
    




#### Import needed packages

In [1]:
import torch, torch.utils.data as data, torch.nn as nn
import torchvision,torchvision.transforms as transforms, torchvision.datasets as datasets

#### Define CNN

In [2]:
class Lenet5(torch.nn.Module):
    def __init__(self):
        super(Lenet5, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 6, (5,5))
        self.conv2 = torch.nn.Conv2d(6, 10, (5,5))
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.fc1 = torch.nn.Linear(160, 120)
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc_class = torch.nn.Linear(84, 10)

    def forward(self, X):
        X=self.pool(torch.tanh(self.conv1(X)))
        X=self.pool(torch.tanh(self.conv2(X)))
        X=X.view([X.size()[0],160])
        X=torch.tanh(self.fc1(X))
        X=torch.tanh(self.fc2(X))
        return self.fc_class(X)

#### Load datasets

In [3]:
trainloader = data.DataLoader(datasets.MNIST(
    root='./data', train=True, download=True, transform=transforms.ToTensor()), batch_size=256, shuffle=False)
testloader = data.DataLoader(datasets.MNIST(
    root='./data', train=False, download=True, transform=transforms.ToTensor()), batch_size=10000)

#### Exercise: make model classify images into odd and even:

Implement required methods of class OddEvenDataset so that MNIST classified images into odd-digit and even digit.

In [4]:
class OddEvenDataset():
    def __init__(self,dataset):
        self.dataset=dataset

    def __len__(self):
        raise NotImplementedError()

    def __getitem__(self,n):
        raise NotImplementedError()
    
train_dataset= OddEvenDataset(datasets.MNIST(
    root='./data', train=True, download=True, transform=transforms.ToTensor()))

test_dataset= OddEvenDataset(datasets.MNIST(
    root='./data', train=False, download=True, transform=transforms.ToTensor()))

#trainloader = data.DataLoader(train_dataset, batch_size=256, shuffle=False)
#testloader = data.DataLoader(test_dataset, batch_size=10000)

#### Train CNN

In [5]:
net=Lenet5()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=.01, momentum=0.9)

for epoch in range(30):
    for batch_idx, (X,y_gt) in enumerate(trainloader):
        optimizer.zero_grad() # clear gradients
        y_out = net(X)
        loss = criterion(y_out, y_gt) # compute error
        loss.backward() # compute weiht updates with respect to error
        optimizer.step() # apply weight updates
    inputs, targets = next(iter(testloader))
    outputs = net(inputs)
    _, predicted = outputs.max(1)
    print(("Epoch {}:  Validation Accuracy {:0.2f}%".format(epoch+1,predicted.eq(targets).sum().item()/100.0)))

Epoch 1:  Validation Accuracy 0.8963%
Epoch 2:  Validation Accuracy 0.9492%
Epoch 3:  Validation Accuracy 0.9624%
Epoch 4:  Validation Accuracy 0.9687%
Epoch 5:  Validation Accuracy 0.9731%


KeyboardInterrupt: 